In [1]:
import numpy as np
import pandas as pd

# Advanced Applied Econometrics

# Labor Problem Set 1

In order to understand the principle of solving finite horizon models like the one in KW97 via backward induction, this problem set requires you to solve a simple one-person, three-period decision problem.

Robinson has to ensure his survival for the next 3 days. He has to work the land in order to produce food. Each day he can choose to work for 8, 12, or 16 hours. He can enjoy the remaining part of the day, such that his leisure time is given by $l_t = 24 - h_t$, where $h_t$ denotes his choice of hours of work.

On Day 1 Robinson is completely unskilled and no matter how many hours he works, he gets 1 unit of produce. Working more  than 8 hours a day allows Robinson to learn how to take better care of the crops, such that he is able to produce more. In particular, for every additional 4 hours of work on either day Robinson levels up his farming skills by 1 unit. E.g., if Robinson works for 16 hours on Day 1, his skills/experience on Day 2 will be 2; if Robinson then also works 12 hours on Day 2, his skills/experience on Day 3 would be 3.  Produce $c_t$ is a function of experience $e_t$:

\begin{equation}
c_t = {(1+ 0.25e_t)}^2
\tag{1}
\end{equation}

Robinson gains utility from consumption and leisure:

\begin{equation}
u(c_t; l_t) = {c_t}^\alpha {l_t}^\beta
\tag{2}
\end{equation}

with $ \alpha = \beta  = 0.5 $

## Questions A:

1. If Robinson had to survive only for 1 day, how many hours would he work?
How much utility would he get? What incentive does Robinson have to make a different choice on Day 1 if he has to survive for 3 days?

2. Is Robinson risk averse?

3. Are consumption and leisure substitutes for Robinson?

4. What are the state variables of the model? How many states are there? Write down a state space representation as a matrix with dimensions *number of states* by *number of state variables*.

5. In dynamic programming problems like this one, the agent makes choices based on the choice specific value functions. These are given by the flow utility plus the continuation value. Assume that continuation values on the last day are zero. What would be optimal number of hours of work for Robinson on Day 3?

6. Note that the flow utilities you calculated in order to arrive to the previous answer are the Day 2 continuation values. What would be optimal numbers of hours to work for Robinson on Day 2? And Day 1?

### See slides for solution

## Question B:

Solve and simulate the model:

- you have found the solution of the model when you have calculated the choice specific value functions at each state
- your simulation output should be a matrix or a matrix-like object with columns "Day", "Choice", and "Choice Specific Value Function"
- *(bonus) your program is general and scalable if you can use the exact same code in order to solve Robinson's survival problem for 30 days (instead of 3)*.

In [2]:
def create_state_space(options):
    """Creates state space for the Robinson Crusoe problem.

    Args:
        options (dict): A dictionary with key parameters for the construction of the model.

    Returns:
        states (np.array): A 2d array of shape (num_states, num_state_variables)
         with all possible states.
        indexer (np.array): A 2d array of shape (num_periods, max_possible_exp)
            with the index of each state in the states array.
    """
    num_periods = options["num_periods"]
    max_possible_exp = num_periods * 2 + 1
    indexer = np.full(shape=(num_periods, max_possible_exp), fill_value=-99, dtype=int)
    states = []
    state_ind = 0
    for period in range(num_periods):
        possible_exp_levels = range(period * 2 + 1)
        for exp_level in possible_exp_levels:
            states += [[period, exp_level]]
            indexer[period, exp_level] = state_ind
            state_ind += 1
    return np.array(states), indexer


def utility_function(hour_choice, exp_level, params):
    """Calculates the flow utility of a given choice dependent on the experience level.

    Args:
        hour_choice (int): The number of hours worked.
        exp_level (int): The experience level.
        params (dict): A dictionary with the parameterization for the model.

    Returns:
        utility (float): The flow utility of the choice.
    """
    consumption = (1 + 0.25 * exp_level) ** 2
    utility_consumption = consumption ** params["alpha"]
    utility_leisure = (24 - hour_choice) ** params["alpha"]
    utility = utility_consumption * utility_leisure
    return utility


def state_next_period(state, hour_choice):
    """Calculates the state in the next period given the current state and the choice.

    Args:
        state (list): A list with the current period and experience level.
        hour_choice (int): The number of hours worked.

    Returns:
        state_next (np.array): A 1d array with the state (period and experience level)
         in the next period.
    """
    period, exp_level = state
    if hour_choice == 8:
        exp_level_next = exp_level
    elif hour_choice == 12:
        exp_level_next = exp_level + 1
    elif hour_choice == 16:
        exp_level_next = exp_level + 2
    else:
        raise ValueError("Invalid choice")
    return np.array([period + 1, exp_level_next])


def backwards_induction(states, indexer, options, params):
    """Calculates the choice specific values for each state in the state space.

    Args:
        states (np.array): A 2d array of shape (num_states, num_state_variables)
         with all possible states.
        indexer (np.array): A 2d array of shape (num_periods, max_possible_exp)
            with the index of each state in the states array.
        options (dict): A dictionary with key parameters for the construction of the model.
        params (dict): A dictionary with the parameterization for the model.

    Returns:
        choice_specific_values (np.array): A 2d array of shape (num_states, num_choices)
            with the choice specific values for each state.
    """
    num_periods = options["num_periods"]
    num_choices = len(options["choices"])
    num_states = states.shape[0]
    choice_specific_values = np.zeros((num_states, num_choices), dtype=float)
    for period in range(num_periods - 1, -1, -1):
        states_period = states[states[:, 0] == period]
        for state in states_period:
            current_state_index = indexer[state[0], state[1]]
            for choice in range(num_choices):
                hours_worked = options["choices"][choice]
                exp_level = state[1]
                utility = utility_function(hours_worked, exp_level, params)
                if period == num_periods - 1:
                    continuation_value = 0
                else:
                    state_next = state_next_period(state, hours_worked)
                    state_next_index = indexer[state_next[0], state_next[1]]
                    # Get the maximum continuation value of next period
                    continuation_value = choice_specific_values[
                        state_next_index, :
                    ].max()

                choice_specific_values[current_state_index, choice] = (
                    utility + continuation_value
                )
    return choice_specific_values


def solve_model(options, params):
    """Solves the model and returns the state space, indexer and choice specific values.

    Args:
        options (dict): A dictionary with key parameters for the construction of the model.
        params (dict): A dictionary with the parameterization for the model.

    Returns:
        states (np.array): A 2d array of shape (num_states, num_state_variables)
         with all possible states.
        indexer (np.array): A 2d array of shape (num_periods, max_possible_exp)
            with the index of each state in the states array.
        choice_specific_values (np.array): A 2d array of shape (num_states, num_choices)
            with the choice specific values for each state.
    """
    states, indexer = create_state_space(options)
    choice_specific_values = backwards_induction(states, indexer, options, params)
    return states, indexer, choice_specific_values


def simulate_model(options, params):
    """Simulates the model and returns a dataframe with the optimal choices, value functions
        and the level of experience at each day.

    Args:
        options (dict): A dictionary with key parameters for the construction of the model.
        params (dict): A dictionary with the parameterization for the model.

    Returns:
        df (pd.DataFrame): Dataframe with the simulated data.
    """
    states, indexer, choice_specific_values = solve_model(options, params)

    num_periods = options["num_periods"]

    # Generate empty dataframe with a row for each day
    index = pd.Index(range(num_periods), name="day")
    df = pd.DataFrame(index=index, columns=["choice", "value_function", "exp_level"])
    # There is only one initial state
    current_state = [0, 0]
    for period in range(num_periods):
        current_state_index = indexer[current_state[0], current_state[1]]
        choice = choice_specific_values[current_state_index, :].argmax()
        value_function = choice_specific_values[current_state_index, choice]
        hour_choice = options["choices"][choice]
        # Save optimal data in DataFrame
        df.loc[period, "choice"] = hour_choice
        df.loc[period, "value_function"] = value_function
        df.loc[period, "exp_level"] = current_state[1]

        current_state = state_next_period(current_state, hour_choice)
    return df

In [3]:
options_robinson = {
    "num_periods": 3,
    "choices": [8, 12, 16],
}

params_robinson = {"alpha": 0.5}

simulate_model(options_robinson, params_robinson)

,choice,value_function,exp_level
day,,,
0,16,15.071068,0
1,16,12.242641,2
2,8,8.0,4
